In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import utility
import math
from time import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.metrics import classification_report

In [2]:
def load_column():
    df = pd.read_csv('./datasets/column_2C.dat', sep=" ", names=['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis','target'])
    df['target']=np.where(df['target']=='AB',1,0)
    return df

def load_blood_transfusion():
    df = pd.read_csv('./datasets/blood_transfusion.csv')
    return df

def load_ionosphere():
    df = pd.read_csv('./datasets/Ionosphere.csv')
    return df

def load_parkinson():
    df = pd.read_csv('./datasets/parkinsons.csv')
    return df

def load_pima():
    df = pd.read_csv('./datasets/diabetes.csv')
    df.rename(columns={'Outcome': 'target'}, inplace=True)
    return df

def load_glass():
    df = pd.read_csv('./datasets/glass.csv')
    # Avoid to_categorical error due to missing class '4'
    unique_labels = sorted(df['target'].unique())  # Ensure sorted order
    label_map = {original: new for new, original in enumerate(unique_labels)}
    
    df['target'] = df['target'].map(label_map)
    return df

def load_climate():
    df = pd.read_csv('./datasets/climate_model_simulation_crashes.csv')
    return df

def load_modeling():
    df = pd.read_csv('./datasets/User_Knowledge_Modeling.csv')
    unique_labels = sorted(df['target'].unique())  # Ensure sorted order
    label_map = {original: new for new, original in enumerate(unique_labels)}
    
    df['target'] = df['target'].map(label_map)
    return df

def load_banknote():
    df = pd.read_csv('./datasets/banknote_authentication.csv')
    return df

def load_sonar():
    df = pd.read_csv('./datasets/sonar.csv')
    return df

In [3]:
#SklearnDatasets
df = load_sonar()
dir_path = 'Sonar'

In [4]:
df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f52,f53,f54,f55,f56,f57,f58,f59,f60,target
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,0
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,0
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,0
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,0


In [5]:
scaler = MinMaxScaler()
scaler.fit(df.values[:, :-1])
scaled_df = scaler.transform(df.values[:, :-1])
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)

0.0 1.0000000000000002


In [6]:
scaled_df.shape

(208, 60)

In [7]:
df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
#targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
#df_scaled['target'] = targets
targets = df['target'].values
df_scaled['target'] = df['target']
num_classes = len(df_scaled['target'].unique())
display(df_scaled)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f52,f53,f54,f55,f56,f57,f58,f59,f60,target
0,0.136431,0.156451,0.135677,0.035426,0.224956,0.237571,0.407468,0.340904,0.449282,0.285714,...,0.027104,0.155844,0.435673,0.149660,0.417949,0.502841,0.185355,0.245179,0.060046,1
1,0.323009,0.221603,0.272011,0.150024,0.283033,0.666756,0.574405,0.755458,0.483045,0.394537,...,0.108417,0.218182,0.111111,0.199546,0.479487,0.389205,0.105263,0.140496,0.087760,1
2,0.182153,0.246892,0.356110,0.243699,0.230028,0.585327,0.648810,0.819405,0.817859,0.869584,...,0.319544,0.418182,0.248538,0.394558,0.615385,0.889205,0.368421,0.258953,0.166282,1
3,0.062684,0.070724,0.199737,0.034950,0.034999,0.071486,0.288149,0.269239,0.077447,0.164593,...,0.161198,0.080519,0.409357,0.179138,0.176923,0.133523,0.093822,0.107438,0.256351,1
4,0.550885,0.282898,0.153088,0.079886,0.132640,0.147003,0.318182,0.531863,0.516659,0.621479,...,0.032810,0.127273,0.277778,0.235828,0.028205,0.196023,0.102975,0.292011,0.203233,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.126844,0.145735,0.050263,0.028293,0.082678,0.410642,0.539773,0.361411,0.333629,0.367653,...,0.154066,0.241558,0.552632,0.061224,0.248718,0.176136,0.256293,0.528926,0.348730,0
204,0.227139,0.040720,0.092970,0.120304,0.175755,0.230046,0.258929,0.212348,0.141419,0.291863,...,0.075606,0.228571,0.365497,0.129252,0.151282,0.088068,0.066362,0.168044,0.140878,0
205,0.373894,0.184741,0.054205,0.055635,0.072026,0.287288,0.331169,0.247630,0.175181,0.345488,...,0.216833,0.062338,0.119883,0.126984,0.217949,0.389205,0.308924,0.209366,0.057737,0
206,0.212389,0.148736,0.156045,0.130766,0.025361,0.336469,0.387446,0.235502,0.276914,0.320463,...,0.111270,0.106494,0.339181,0.068027,0.079487,0.088068,0.173913,0.096419,0.096998,0


In [8]:
np.random.seed(50)
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
n_neurons = X_train.shape[1]
n_hidden_layers = 1
print(f'n_hidden_layers: {n_hidden_layers}, n_neurons {n_neurons}')
model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    layers.Dense(n_neurons, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model_path = f'new_models/{dir_path}.keras'
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
ck = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)
start = time()
model.fit(X_train,y_train, epochs=400, batch_size=32,validation_data=(X_test, y_test), verbose=1, callbacks=[ck, es])
print(f'Training time: {time()-start}')
model = tf.keras.models.load_model(model_path)
print('Training Results')
model.evaluate(X_train, y_train, verbose=2)
print('Test Results')
model.evaluate(X_test, y_test, verbose=2)
model.save(f'new_models/{dir_path}.h5', save_format='h5')

n_hidden_layers: 1, n_neurons 60


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           122 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,782 (14.77 KB)

 Trainable params: 3,782 (14.77 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/400
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.5152 - loss: 0.7161 - val_accuracy: 0.5769 - val_loss: 0.6726
Epoch 2/400
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.5721 - loss: 0.6525 - val_accuracy: 0.5962 - val_loss: 0.6524
Epoch 3/400
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6234 - loss: 0.6454 - val_accuracy: 0.6090 - val_loss: 0.6448
Epoch 4/400
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.6442 - loss: 0.6188 - val_accuracy: 0.6218 - val_loss: 0.6437
Epoch 5/400
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.7139 - loss: 0.6131 - val_accuracy: 0.6090 - val_loss: 0.6445
Epoch 6/400
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6571 - loss: 0.6195 - val_accuracy: 0.6154 - val_loss: 0.6422
Epoch 7/400
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6571 - loss: 0.6086 - val_accuracy: 0.6154 - val_loss: 0.6376
Epoch 8/400
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7604 - loss: 0.5955 - val_accuracy: 0.6154 - val

In [10]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  

y_test_classes = np.argmax(y_test, axis=1)

report = classification_report(y_test_classes, y_pred_classes,digits=4)
print(report)

# Save the report to a text file
with open(f'new_models/{dir_path}_report.txt', 'w') as f:
    f.write(report)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
              precision    recall  f1-score   support

           0     0.7900    0.9518    0.8634        83
           1     0.9286    0.7123    0.8062        73

    accuracy                         0.8397       156
   macro avg     0.8593    0.8321    0.8348       156
weighted avg     0.8548    0.8397    0.8366       156

